In [6]:
# loading the API keys in environment variables
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [5]:
!pip install -r ./requirements.txt -q

In [11]:
pip install langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open('actresses.txt') as f:
    actresses = f.read()


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len
)

In [8]:
chunks = text_splitter.create_documents([actresses])
print(chunks[2])
print(chunks[12].page_content)
print(f'This file is made up of  {len(chunks)} chunks')

page_content='generation, Streep has mesmerized audiences with her unparalleled talent and versatility for over'
to fully embody characters, coupled with her dedication to her craft, has made her a beacon of
This file is made up of  51 chunks


In [9]:
def calculate_cost(text):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in text])
    print(f'Total Tokens: {total_tokens}')
    print(f'This costs in total in GBP: {total_tokens / 1000 * 0.0004:.6f}')
    
calculate_cost(chunks)

Total Tokens: 956
This costs in total in GBP: 0.000382


In [17]:
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

OPENAIAPIKEY=os.environ.get("OPENAI_API_KEY")
embeddings = OpenAIEmbeddings(openai_api_key=OPENAIAPIKEY)

vector = embeddings.embed_query('cats')
vector

vector2 = embeddings.embed_query('dogs')
vector2

vector3 = embeddings.embed_query('rabbits')
vector3


vector_real = embeddings.embed_query(chunks[0].page_content)
vector_real
print(vector_real)
# see below

[-0.01520228116530352, -0.02086846544167177, -0.011391805559839504, -0.04366528644083603, 0.006745929874158872, 0.016840059420991357, -0.017011761777267424, 0.009476660813154845, -0.02049864469543945, -0.03280839718111713, 0.01592871557225065, 0.014938123622469374, 0.04366528644083603, -0.01480604438539101, -0.0003696145455921787, -0.0006554415492622664, 0.007403022192695505, 0.026111997835769903, 0.01273240623059296, -0.029638504804462208, -0.002213972635232118, 0.011180479153043155, 0.0011705493165824006, 0.02059110081332011, 0.014145650062644354, -0.013439028612279948, 0.008426633718915727, -0.023074183764870766, -0.0002854142531438348, -0.01217107110282443, -0.007779447413009164, -0.02430251815512858, -0.01135878551773927, -0.03317822165263977, -0.01953446877933836, -0.027208253920271933, -0.018002353727048698, -0.041525606339192354, 0.01151727985717524, -0.006531301463152499, -0.0018325947578493916, 0.020551476762799826, -0.025002535597072936, -0.017289128268264245, -0.01359091894

In [33]:
import os
from pinecone import Pinecone

from langchain.vectorstores import Pinecone


from pinecone import Pinecone

# new way to instantiate the pinecone class
pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))



NameError: name 'embeddings' is not defined

In [43]:
# create a pinecone index
# if using free plan - limited to one index and one project
# delete all indexes first

import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

OPENAIAPIKEY=os.environ.get("OPENAI_API_KEY")

from langchain.vectorstores import Pinecone


from pinecone import Pinecone

# new way to instantiate the pinecone class
pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))

# delete all indexes
indexes = pc.list_indexes()
print(indexes)
# delete all indexes in the list
for i in indexes:
    print('Deleting all indexes ...', end='')
    pinecone.delete_index(i)
    print('Done')

{'indexes': []}


In [57]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

OPENAIAPIKEY=os.environ.get("OPENAI_API_KEY")
from pinecone import Pinecone, PodSpec
    
pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))

index_name = 'actresses'
if index_name not in pc.list_indexes():
    print(f'Creating index {index_name}...')
    pc.create_index(index_name, dimension=1536, metric='cosine',  spec=PodSpec(
    environment="gcp-starter"
   
  ))
    print('Done!')

Creating index actresses...
Done!


In [66]:
# upload the vectors to pinecone
# from langchain.vectorstores import Pinecone

#pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))
vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)

# we can see this in the pinecone database 

In [68]:
query = '''Where was Vera Farmiga born'''
result = vector_store.similarity_search(query)
print(result)

[Document(page_content='Vera Farmiga, born on August 6, 1973, in Clifton, New Jersey, is an actress known for her'), Document(page_content='Apart from her acting prowess, Farmiga is known for her directorial debut, "Higher Ground" (2011),'), Document(page_content="critical acclaim. Farmiga's ability to portray characters with depth and nuance has made her a"), Document(page_content='has made her a sought-after actress, and her filmography includes standout performances in movies')]


In [79]:
for x in result:
    print(x.page_content)
    print('-' * 100)

Vera Farmiga, born on August 6, 1973, in Clifton, New Jersey, is an actress known for her
----------------------------------------------------------------------------------------------------
Apart from her acting prowess, Farmiga is known for her directorial debut, "Higher Ground" (2011),
----------------------------------------------------------------------------------------------------
critical acclaim. Farmiga's ability to portray characters with depth and nuance has made her a
----------------------------------------------------------------------------------------------------
has made her a sought-after actress, and her filmography includes standout performances in movies
----------------------------------------------------------------------------------------------------


In [80]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import openai

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 33})

chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [76]:
query = 'Where was Vera Farmiga born'
answer = chain.run(query)
print(answer)


C:\Users\emily\miniconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Vera Farmiga was born in Clifton, New Jersey.
